<a href="https://colab.research.google.com/github/AISushilRaj/CODSOFT/blob/main/SUSHIL_RAJCodSoft_TASK_3_SPAM_SMS_DETECTION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
uploaded = files.upload()

Saving spamdata.xls to spamdata.xls


In [2]:
filename= list(uploaded.keys())[0]

In [3]:
import pandas as pd
import io
df = pd.read_excel(io.BytesIO(uploaded[filename]))

In [4]:
df=df.drop(columns=['Unnamed: 2','Unnamed: 3','Unnamed: 4'])

In [5]:
df.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [6]:
df = df[['v1', 'v2']].rename(columns={'v1': 'label', 'v2': 'message'})

In [7]:
df['label'] = df['label'].map({'ham': 0, 'spam': 1})

In [8]:
df['message'] = df['message'].astype(str)

In [9]:
df.head()

,label,message
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

# Split the data into training and test sets (80-20 split)
X_train, X_test, y_train, y_test = train_test_split(df['message'], df['label'], test_size=0.2, random_state=42)

# Convert text into TF-IDF features
vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Train classifiers
models = {
    "Naive Bayes": MultinomialNB(),
    "Logistic Regression": LogisticRegression(max_iter=200),
    "SVM": SVC(kernel='linear')
}

# Train and evaluate models
results = {}
results_data = []
for name, model in models.items():
    model.fit(X_train_tfidf, y_train)
    y_pred = model.predict(X_test_tfidf)
    acc = accuracy_score(y_test, y_pred)
    results[name] = {
        "Accuracy": acc,
        "Report": classification_report(y_test, y_pred)
    }
    results_data.append({
        "Model": name,
        "Accuracy": acc
    })

# Create a DataFrame from the results data
results_df = pd.DataFrame(results_data)


results
display(results_df)

,Model,Accuracy
0,Naive Bayes,0.972197
1,Logistic Regression,0.957848
2,SVM,0.979372


In [11]:
model = LogisticRegression(max_iter=200)  # Or any other model from your 'models' dictionary
model.fit(X_train_tfidf, y_train)
y_pred = model.predict(X_test_tfidf)
y_prob = model.predict_proba(X_test_tfidf)[:, 1]  # Probability of class 1 (spam)


# Create a DataFrame with predictions for each SMS
predictions_df = pd.DataFrame({
    'SMS Message': X_test,
    'Actual Label': y_test,
    'Predicted Label': y_pred,
    'Spam Probability': y_prob
})

# Display the DataFrame
display(predictions_df)

,SMS Message,Actual Label,Predicted Label,Spam Probability
3245,"Funny fact Nobody teaches volcanoes 2 erupt, t...",0,0,0.086158
944,I sent my scores to sophas and i had to do sec...,0,0,0.098277
1044,We know someone who you know that fancies you....,1,0,0.233569
2484,Only if you promise your getting out as SOON a...,0,0,0.031605
812,Congratulations ur awarded either å£500 of CD ...,1,1,0.900261
...,...,...,...,...
4264,&lt;DECIMAL&gt; m but its not a common car he...,0,0,0.020945
2439,Rightio. 11.48 it is then. Well arent we all u...,0,0,0.063366
5556,Yes i have. So that's why u texted. Pshew...mi...,0,0,0.050006
4205,"Get the door, I'm here",0,0,0.074043
